In [10]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import os
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_col

# Downloading and Assigning Regions to Table A2

In [14]:
# This is the HCD data from 2018-2023, downloaded Jan 2025

hcd_data = pd.read_csv('/home/jovyan/bac/data/tablea2.csv')
hcd_data.head()

/tmp/ipykernel_136/2713971221.py:3: DtypeWarning: Columns (3,16,25,33,36,37,42,43,49,50) have mixed types. Specify dtype option on import or set low_memory=False.
  hcd_data = pd.read_csv('/home/jovyan/bac/data/tablea2.csv')


,JURIS_NAME,CNTY_NAME,YEAR,PRIOR_APN,APN,STREET_ADDRESS,PROJECT_NAME,JURS_TRACKING_ID,UNIT_CAT,TENURE,...,NO_FA_DR,TERM_AFF_DR,DEM_DES_UNITS,DEM_OR_DES_UNITS,DEM_DES_UNITS_OWN_RENT,DENSITY_BONUS_TOTAL,DENSITY_BONUS_NUMBER_OTHER_INCENTIVES,DENSITY_BONUS_INCENTIVES,DENSITY_BONUS_RECEIVE_REDUCTION,NOTES
0,ADELANTO,San Bernardino,2018,NaN,312846105,15824 BELLFLOWER ST,JOSE & ODULIA LOPEZ,NaN,SFD,Owner,...,0,NaN,0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
1,ADELANTO,San Bernardino,2018,NaN,312878124,11939 GAINES CT,"FRONTIER HOMES II, LLC",NaN,SFD,Owner,...,0,NaN,0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
2,ADELANTO,San Bernardino,2018,NaN,312878125,11931 GAINES CT,"FRONTIER HOMES II, LLC",NaN,SFD,Owner,...,0,NaN,0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
3,ADELANTO,San Bernardino,2018,NaN,312878126,11925 GAINES CT,"FRONTIER HOMES II, LLC",NaN,SFD,Owner,...,0,NaN,0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
4,ADELANTO,San Bernardino,2018,NaN,312878127,11913 GAINES CT,"FRONTIER HOMES II, LLC",NaN,SFD,Owner,...,0,NaN,0,0.0,0.0,0.0,0.0,NaN,NaN,NaN


In [15]:
# Define regions in California
regions = {
    "Bay Area": ["Alameda", "Contra Costa", "Marin", "Napa", "San Francisco", "San Mateo", "Santa Clara", "Solano", "Sonoma", "Santa Cruz"],
    "Central Region": ["Fresno", "Kern", "Kings", "Madera", "Mariposa", "Merced", "Monterey", "San Benito", "San Luis Obispo", "Stanislaus", "Tulare", "Tuolumne"],
    "Inland Deserts Region": ["Imperial", "Inyo", "Mono", "Riverside", "San Bernardino"],
    "North Central Region": ["Sacramento", "San Joaquin", "El Dorado", "Lake", "Nevada", "Placer", "Sierra", "Sutter", "Plumas", "Yolo", "Yuba",
                             "Alpine", "Amador", "Calaveras", "Butte", "Colusa", "Glenn"],
    "Northern Region": ["Del Norte", "Humboldt", "Lassen", "Mendocino", "Modoc", "Shasta", "Siskiyou", "Tehama", "Trinity"],
    "South Coast Region": ["Los Angeles", "Orange", "San Diego", "Santa Barbara", "Ventura"]
}

# Map counties to region
def get_region(county):
    for region, counties in regions.items():
        if county in counties:
            return region
    return "Other California"  # Default category for any unmatched counties

# Apply function to create Region column
hcd_data['Region'] = hcd_data['CNTY_NAME'].apply(get_region)

In [16]:
# Map UNIT_CAT to descriptive names to create Type column
unit_cat_mapping = {
    "SFD": "Single-Family Detached Unit",
    "SFA": "Single-Family Attached Unit",
    "MH": "Mobile Home Unit",
    "2 to 4": "2-, 3-, and 4-Plex Units per Structure",
    "ADU": "Accessory Dwelling Unit",
    "5+": "5 or More Units per Structure"
}

hcd_data['Type'] = hcd_data['UNIT_CAT'].map(unit_cat_mapping)

hcd_data

,JURIS_NAME,CNTY_NAME,YEAR,PRIOR_APN,APN,STREET_ADDRESS,PROJECT_NAME,JURS_TRACKING_ID,UNIT_CAT,TENURE,...,DEM_DES_UNITS,DEM_OR_DES_UNITS,DEM_DES_UNITS_OWN_RENT,DENSITY_BONUS_TOTAL,DENSITY_BONUS_NUMBER_OTHER_INCENTIVES,DENSITY_BONUS_INCENTIVES,DENSITY_BONUS_RECEIVE_REDUCTION,NOTES,Region,Type
0,ADELANTO,San Bernardino,2018,NaN,312846105,15824 BELLFLOWER ST,JOSE & ODULIA LOPEZ,NaN,SFD,Owner,...,0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,Inland Deserts Region,Single-Family Detached Unit
1,ADELANTO,San Bernardino,2018,NaN,312878124,11939 GAINES CT,"FRONTIER HOMES II, LLC",NaN,SFD,Owner,...,0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,Inland Deserts Region,Single-Family Detached Unit
2,ADELANTO,San Bernardino,2018,NaN,312878125,11931 GAINES CT,"FRONTIER HOMES II, LLC",NaN,SFD,Owner,...,0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,Inland Deserts Region,Single-Family Detached Unit
3,ADELANTO,San Bernardino,2018,NaN,312878126,11925 GAINES CT,"FRONTIER HOMES II, LLC",NaN,SFD,Owner,...,0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,Inland Deserts Region,Single-Family Detached Unit
4,ADELANTO,San Bernardino,2018,NaN,312878127,11913 GAINES CT,"FRONTIER HOMES II, LLC",NaN,SFD,Owner,...,0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,Inland Deserts Region,Single-Family Detached Unit
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
646935,YUCCA VALLEY,San Bernardino,2023,NaN,0598-601-10,58855 MEREDITH CT,NaN,2021-2239,SFD,Owner,...,0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,Inland Deserts Region,Single-Family Detached Unit
646936,YUCCA VALLEY,San Bernardino,2023,NaN,0598-601-12,58871 MEREDITH CT,NaN,2021-2266,SFD,Owner,...,0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,Inland Deserts Region,Single-Family Detached Unit
646937,YUCCA VALLEY,San Bernardino,2023,NaN,0601-021-43,57750 PAXTON RD,NaN,2023-333,SFD,Owner,...,0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,Inland Deserts Region,Single-Family Detached Unit
646938,YUCCA VALLEY,San Bernardino,2023,NaN,0601-081-66,59045 WILCOX LN,NaN,2022-905,SFD,Owner,...,0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,Inland Deserts Region,Single-Family Detached Unit


In [17]:
hcd_data.to_csv('/home/jovyan/bac/data/hcd_data.csv')